In [18]:
import numpy as np
from redisvl.index import SearchIndex
from redis import Redis
from redisvl.query import VectorQuery
from redisvl.redis.utils import buffer_to_array

In [19]:
client = Redis.from_url("redis://localhost:6379")

In [20]:
schema = {
    "index": {
        "name": "user_simple",
        "prefix": "user_simple_docs", 
        "storage_type": "hash"
    },
    "fields": [
        {"name": "user", "type": "tag"}, 
        {
            "name": "user_embedding",
            "type": "vector", 
            "attrs": {
                "dims": 3, 
                "distance_metric": "cosine", 
                "algorithm": "flat",  
                "datatype": "float32" 
            }
        }
    ]
}
index = SearchIndex.from_dict(schema, redis_client=client, validate_on_load=True)

index.create(overwrite=True)

17:22:49 redisvl.index.index INFO   Index already exists, overwriting.


In [21]:
data = [
    {
        'user': 'john',
        'user_embedding': np.array([0.1, 0.1, 0.5], dtype=np.float32).tobytes()  # hash存储时,向量必须以字节字符串的形式存储
    },
    {
        'user': 'mary',
        'user_embedding': np.array([0.2, 0.1, 0.5], dtype=np.float32).tobytes()
    }
]

keys = index.load(data)

In [22]:
query_vector = VectorQuery(
    vector=[0.1, 0.1, 0.5],
    vector_field_name="user_embedding",
    return_fields=["user", "age", "job", "vector_distance"],
    num_results=3
)
results_vector = index.query(query_vector)
for i in results_vector:
    print(i)
    id_ = i["id"]
    key_type = client.type(id_)
    if key_type == b"hash":
        hash_data = client.hgetall(id_)
        print("向量的字节字符串形式: ", hash_data[b'user_embedding'])
        embedding = buffer_to_array(hash_data[b'user_embedding'], dtype='float32')
        print("向量的数组像是: ", embedding)

{'id': 'user_simple_docs:01K38HCVEMTR5E8SHPQF9KBTSW', 'vector_distance': '0', 'user': 'john'}
向量的字节字符串形式:  b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'
向量的数组像是:  [0.10000000149011612, 0.10000000149011612, 0.5]
{'id': 'user_simple_docs:01K38HCZSX46RDBHP1QS30EB7C', 'vector_distance': '0', 'user': 'john'}
向量的字节字符串形式:  b'\xcd\xcc\xcc=\xcd\xcc\xcc=\x00\x00\x00?'
向量的数组像是:  [0.10000000149011612, 0.10000000149011612, 0.5]
{'id': 'user_simple_docs:01K38HCVEMTR5E8SHPQF9KBTSX', 'vector_distance': '0.0161802768707', 'user': 'mary'}
向量的字节字符串形式:  b'\xcd\xccL>\xcd\xcc\xcc=\x00\x00\x00?'
向量的数组像是:  [0.20000000298023224, 0.10000000149011612, 0.5]


In [23]:
index.delete()